# Mapping antenna data to geographic regions

Input parameters for this notebook:

In [ ]:
regions_geojson_file = "/home/dominic/Downloads/Limite_Bairro_clean.geojson"
antennas_csv_file = "antenna_pos.csv"

The `mobilephonenetworkanalysis` package is a support library shipped by this package. It can be install with `python -m pip install .` from the repository root directory.

In [ ]:
import geojson
import itertools
import mobilephonenetworkanalysis
import numpy as np
import pyproj
import shapely.geometry as geo
import scipy.spatial as spatial

Read the regions both as GeoJSON and a list of shapely geometry objects:

In [ ]:
with open(regions_geojson_file) as f:
    data = geojson.load(f)

In [ ]:
shapely_polygons = [geo.shape(f["geometry"]) for f in data["features"]]

Read the antenna positions both as `np.array` and shapely geometry objects:

In [ ]:
antennas = np.genfromtxt(antennas_csv_file, delimiter=",")

In [ ]:
projection = pyproj.Transformer.from_crs(
    "EPSG:4326", "EPSG:29193", always_xy=False
).transform
antennas = np.apply_along_axis(lambda row: projection(*row), 1, antennas)

Calculate a Voronoi tesselation and transform to shapely:

In [ ]:
vor = spatial.Voronoi(antennas)
regions, vertices = mobilephonenetworkanalysis.voronoi_finite_polygons_2d(vor)
voronoi_polygons = [geo.Polygon([vertices[i] for i in reg]) for reg in regions]

Calculate the transformation matrix that describes the relation ship between antennas and barrios:

In [ ]:
b2b = np.zeros(shape=(len(voronoi_polygons), len(shapely_polygons)))
for (i, vp), (j, barrio) in itertools.product(
    enumerate(voronoi_polygons), enumerate(shapely_polygons)
):
    b2b[i, j] = vp.intersection(barrio).area / barrio.area

We can assert that in this calculation all barrios are fully covered by antennas, whereas the same assertion for `axis=1` does not hold because there are antennas outside of the city:

In [ ]:
assert np.allclose(np.sum(b2b, axis=0), 1.0)